### パラメータ設定


In [0]:
dbutils.widgets.text("cr_creator_input_schema", "default", "クリエイターの入力スキーマ")         # 実際の環境に合わせて変更してください
dbutils.widgets.text("cr_collaborator_input_schema", "default", "コラボレーターの入力スキーマ")  # 実際の環境に合わせて変更してください

dbutils.widgets.text("cr_output_catalog", "", "ノートブック実行者の出力カタログ")                 # システム値：編集しないでください
dbutils.widgets.text("cr_output_schema", "", "ノートブック実行者の出力スキーマ")                  # システム値：編集しないでください

### ID突合して結果を出力カタログに保存

`customer`と`sales`の顧客データをハッシュ化された顧客IDで結合します。  
あなたとコラボレーターが同じハッシュアルゴリズムを使用してメールアドレスを正規化していることを前提とします。

**注意事項**

- あなたは `customer` テーブルをアップロードし、コラボレーターは `sales` テーブルをアップロードします。

In [0]:
# クリエイターの入力スキーマ
creator_schema = dbutils.widgets.get("cr_creator_input_schema")

# コラボレーターの入力スキーマ
collaborator_schema = dbutils.widgets.get("cr_collaborator_input_schema")

# ノートブック実行者の出力カタログ
output_catalog = dbutils.widgets.get("cr_output_catalog")

# ノートブック実行者の出力スキーマ
output_schema = dbutils.widgets.get("cr_output_schema")

In [0]:
if output_catalog and output_schema:
  spark.sql(
    f"""
    CREATE TABLE {output_catalog}.{output_schema}.overlapping_customers_sales
    AS
    SELECT cu.*
    FROM creator.{creator_schema}.customers cu
    INNER JOIN collaborator.{collaborator_schema}.orders od
    ON cu.customerid_sha256 = od.customerid_sha256
    """
  )
else:
    print("出力カタログまたはスキーマが指定されていません。テーブル作成をスキップします。")